In [1]:
import larch as lx
import jax
import jax.numpy as jnp
import xarray as xr

/Users/jeffnewman/LocalGit/larch/larch/numba/model.py:23: UserWarning: 

### larch.numba is experimental, and not feature-complete ###
 the first time you import on a new system, this package will
 compile optimized binaries for your machine, which may take 
 a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
d = lx.examples.MTC(format='dataset')

In [3]:
m = lx.Model(d)
from larch import P, X, PX
m.utility_co[2] = P("ASC_SR2")  + P("hhinc#2") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

m.utility_ca = PX("tottime") + PX("totcost")

m.availability_var = 'avail'
m.choice_ca_var = 'chose'

m.title = "MTC Example 1 (Simple MNL)"
r = m.maximize_loglike(quiet=True)


In [4]:
groupid = np.repeat(
    np.arange(1300), 
    np.random.default_rng(0).choice([3,5], size=1300)
)[:m.dataset.n_cases]

m.groupid = groupid

In [5]:
# dataset = m.dataset.assign_coords(groupid=xr.DataArray(groupid, dims='caseid'))
# dataset

In [6]:
from larch.folding import fold, fold_dataset

In [7]:
# fdataset = fold_dataset(dataset)
# fdataset

In [8]:
# m.dataset = fdataset

In [9]:
m.pf

,value,initvalue,nullvalue,minimum,maximum,holdfast,note
ASC_BIKE,-2.376328,0.0,0.0,-inf,inf,0,
ASC_SR2,-2.178014,0.0,0.0,-inf,inf,0,
ASC_SR3P,-3.725078,0.0,0.0,-inf,inf,0,
ASC_TRAN,-0.670861,0.0,0.0,-inf,inf,0,
ASC_WALK,-0.206775,0.0,0.0,-inf,inf,0,
hhinc#2,-0.002170,0.0,0.0,-inf,inf,0,
hhinc#3,0.000358,0.0,0.0,-inf,inf,0,
hhinc#4,-0.005286,0.0,0.0,-inf,inf,0,
hhinc#5,-0.012808,0.0,0.0,-inf,inf,0,
hhinc#6,-0.009686,0.0,0.0,-inf,inf,0,


In [10]:
ca = jnp.asarray(m.dataset['ca'])
co = jnp.asarray(m.dataset['co'])
ch = jnp.asarray(m.dataset['ch'])
av = jnp.asarray(m.dataset['av'])

params = jnp.asarray(m.pvals)


/Users/jeffnewman/LocalGit/larch/larch/dataset.py:466: UserWarning: no defined CASEID
  warnings.warn("no defined CASEID")


In [11]:
m.dataset

<larch.Dataset>
Dimensions:    (altid: 6, var_ca: 2, groupid: 1241, ingroupid: 5, var_co: 1)
Coordinates:
    alt_names  (altid) <U7 'DA' 'SR2' 'SR3+' 'Transit' 'Bike' 'Walk'
  * altid      (altid) int64 1 2 3 4 5 6
  * var_ca     (var_ca) <U7 'totcost' 'tottime'
  * var_co     (var_co) <U5 'hhinc'
Dimensions without coordinates: groupid, ingroupid
Data variables:
    ca         (groupid, ingroupid, altid, var_ca) float64 70.63 15.38 ... 0.0
    co         (groupid, ingroupid, var_co) float64 42.5 17.5 12.5 ... 0.0 0.0
    ch         (groupid, ingroupid, altid) int64 1 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    av         (groupid, ingroupid, altid) int8 1 1 1 1 1 0 1 ... 0 0 0 0 0 0 0
    caseid     (groupid, ingroupid) int64 1 2 3 4 5 6 7 ... 5028 5029 0 0 0 0
Attributes:
    _altid_:  altid

In [12]:
m.mix_parameter("totcost", "totcost_s")

/Users/jeffnewman/LocalGit/larch/larch/dataset.py:466: UserWarning: no defined CASEID
  warnings.warn("no defined CASEID")


In [13]:
m.pf

,value,initvalue,nullvalue,minimum,maximum,holdfast,note
ASC_BIKE,-2.376328,0.000,0.0,-inf,inf,0,
ASC_SR2,-2.178014,0.000,0.0,-inf,inf,0,
ASC_SR3P,-3.725078,0.000,0.0,-inf,inf,0,
ASC_TRAN,-0.670861,0.000,0.0,-inf,inf,0,
ASC_WALK,-0.206775,0.000,0.0,-inf,inf,0,
hhinc#2,-0.002170,0.000,0.0,-inf,inf,0,
hhinc#3,0.000358,0.000,0.0,-inf,inf,0,
hhinc#4,-0.005286,0.000,0.0,-inf,inf,0,
hhinc#5,-0.012808,0.000,0.0,-inf,inf,0,
hhinc#6,-0.009686,0.000,0.0,-inf,inf,0,


In [14]:
m.unmangle()

/Users/jeffnewman/LocalGit/larch/larch/dataset.py:466: UserWarning: no defined CASEID
  warnings.warn("no defined CASEID")


In [15]:
z = np.asarray(m.pvals).copy()

In [16]:
# m.dataset = fdataset

In [17]:
m.jax_loglike(z)

DeviceArray(-3618.9358, dtype=float32)

In [18]:
m.jax_loglike(z)

DeviceArray(-3618.9358, dtype=float32)

In [19]:
m.jax_d_loglike(z) # 3.39s

DeviceArray([-6.231248e-02,  1.185981e+00,  3.937205e-01,  1.777634e+00, -1.529565e-02,  6.643169e+01,  2.541438e+01,
              1.355640e+02, -3.391693e+00, -1.586962e+00, -3.815842e+03,  1.199365e+04,  6.031855e+01], dtype=float32)

In [20]:
m.jax_d_loglike(z) # 194ms

DeviceArray([-6.231248e-02,  1.185981e+00,  3.937205e-01,  1.777634e+00, -1.529565e-02,  6.643169e+01,  2.541438e+01,
              1.355640e+02, -3.391693e+00, -1.586962e+00, -3.815842e+03,  1.199365e+04,  6.031855e+01], dtype=float32)

In [21]:
m.n_draws = 1000
m.unmangle()

In [22]:
m.jax_d_loglike(z) # 6.39s

/Users/jeffnewman/LocalGit/larch/larch/dataset.py:466: UserWarning: no defined CASEID
  warnings.warn("no defined CASEID")


DeviceArray([-7.973910e-02,  1.050525e+00,  2.904796e-01,  1.706540e+00, -4.598825e-02,  5.801763e+01,  1.865317e+01,
              1.298990e+02, -4.307726e+00, -2.996945e+00, -3.663306e+03,  1.207497e+04,  5.635622e+01], dtype=float32)

In [23]:
m.jax_d_loglike(z) # 1.83s

DeviceArray([-7.973910e-02,  1.050525e+00,  2.904796e-01,  1.706540e+00, -4.598825e-02,  5.801763e+01,  1.865317e+01,
              1.298990e+02, -4.307726e+00, -2.996945e+00, -3.663306e+03,  1.207497e+04,  5.635622e+01], dtype=float32)

In [24]:
m.jax_loglike(z) # 1.44s

DeviceArray(-3618.9158, dtype=float32)

In [25]:
m.jax_loglike(z) # 455ms

DeviceArray(-3618.9158, dtype=float32)

In [26]:
m.groupid = None

In [27]:
m.jax_loglike(z) # 2.0s

DeviceArray(-3622.1353, dtype=float32)

In [28]:
m.jax_loglike(z) #  482ms

DeviceArray(-3622.1353, dtype=float32)

In [29]:
m.groupid = groupid

In [30]:
m.jax_loglike(z) #  2.24s

/Users/jeffnewman/LocalGit/larch/larch/dataset.py:466: UserWarning: no defined CASEID
  warnings.warn("no defined CASEID")


DeviceArray(-3618.9158, dtype=float32)

In [31]:
m.dataset

<larch.Dataset>
Dimensions:    (altid: 6, var_ca: 2, groupid: 1241, ingroupid: 5, var_co: 1)
Coordinates:
    alt_names  (altid) <U7 'DA' 'SR2' 'SR3+' 'Transit' 'Bike' 'Walk'
  * altid      (altid) int64 1 2 3 4 5 6
  * var_ca     (var_ca) <U7 'totcost' 'tottime'
  * var_co     (var_co) <U5 'hhinc'
Dimensions without coordinates: groupid, ingroupid
Data variables:
    ca         (groupid, ingroupid, altid, var_ca) float64 70.63 15.38 ... 0.0
    co         (groupid, ingroupid, var_co) float64 42.5 17.5 12.5 ... 0.0 0.0
    ch         (groupid, ingroupid, altid) int64 1 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    av         (groupid, ingroupid, altid) int8 1 1 1 1 1 0 1 ... 0 0 0 0 0 0 0
    caseid     (groupid, ingroupid) int64 1 2 3 4 5 6 7 ... 5028 5029 0 0 0 0
Attributes:
    _altid_:  altid